In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import os

# Set environment variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'cortex'

# Get keys from the environment
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

if langchain_api_key:
    os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
else:
    raise ValueError("LANGCHAIN_API_KEY is not set in the environment.")

if groq_api_key:
    os.environ['GROQ_API_KEY'] = groq_api_key
else:
    raise ValueError("GROQ_API_KEY is not set in the environment.")

In [3]:
from langchain.document_loaders import WebBaseLoader

# Suppress tokenizer parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

urls = [
    "https://www.webmd.com/a-to-z-guides/malaria",
    "https://www.webmd.com/diabetes/type-1-diabetes",
    "https://www.webmd.com/diabetes/type-2-diabetes",
    "https://www.webmd.com/migraines-headaches/migraines-headaches-migraines",
]

loader = WebBaseLoader(urls, bs_get_text_kwargs={"strip": True})
docs = loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Splitting the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
chunks = text_splitter.split_documents(docs)

# Defining the embedding function
embedding_function = HuggingFaceEmbeddings()

# Creating the vector store
vector_store = Chroma.from_documents(documents=chunks, embedding=embedding_function)

# Setting up the retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [5]:
retriever.invoke("Symptoms of migraine")

[Document(metadata={'description': 'Migraine headaches, also known as migraines, are a type of headache that can cause debilitating pain. Learn more about the types, causes, symptoms, diagnosis, & treatment of migraines.', 'language': 'en', 'source': 'https://www.webmd.com/migraines-headaches/migraines-headaches-migraines', 'title': 'Migraine: Symptoms, Causes, and Treatment'}, page_content='can be caused by many other things. Some typical characteristics of migraine headaches include:The pain is throbbing or pounding and feels worse when you move around.You feel it mostly on one side of your head.You also have at least one of these symptoms: sensitivity to light and/or sound,nausea, and vomiting.Migraine SymptomsFor many people, a migraine episode happens in stages. Symptoms you may have during these stagesinclude:Migraine prodromeHours or days before a headache, about 60% of'),
 Document(metadata={'description': 'Migraine headaches, also known as migraines, are a type of headache tha

In [6]:
from pydantic import BaseModel
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

class VectorStore(BaseModel):
    """A vectorstore contains information about symptoms, treatment, risk factors and other information about malaria, type 1 and type 2 diabetes, and migraines."""
    query: str

class SearchEngine(BaseModel):
    """A search engine for searching other medical information on the web"""
    query: str

router_prompt_template = (
    "You are an expert in routing user queries to either a VectorStore, SearchEngine\n"
    "Use SearchEngine for all other medical queries that are not related to malaria, diabetes, or migraines.\n"
    "The VectorStore contains information on malaria, diabetes, and migraines.\n"
    'Note that if a query is not medically-related, you must output "not medically-related", don\'t try to use any tool.\n\n'
    "query: {query}"
)

llm = ChatGroq(model="llama3-70b-8192", temperature=0)
prompt = ChatPromptTemplate.from_template(router_prompt_template)
question_router = prompt | llm.bind_tools(tools=[VectorStore, SearchEngine])

In [7]:
response = question_router.invoke("What are the symptoms of chest pain?")
"tool_calls" in response.additional_kwargs

True

In [8]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3twh', 'function': {'arguments': '{"query":"chest pain"}', 'name': 'SearchEngine'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 1111, 'total_tokens': 1165, 'completion_time': 0.172056568, 'prompt_time': 0.036108169, 'queue_time': 0.058618312, 'total_time': 0.208164737}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c7c2f4b3-fc2a-4d0b-b8cb-9546ee149cf1-0', tool_calls=[{'name': 'SearchEngine', 'args': {'query': 'chest pain'}, 'id': 'call_3twh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1111, 'output_tokens': 54, 'total_tokens': 1165})

In [9]:
from pydantic import field_validator
from pydantic import BaseModel, Field
from typing import Literal

class Grader(BaseModel):
    "Use this format to give a binary score for relevance check on retrived documents."

    grade: Literal["relevant", "irrelevant"] = Field(
        ...,
        description="The relevance score for the document.\n"
        "Set this to 'relevant' if the given context is relevant to the user's query, or 'irrlevant' if the document is not relevant.",
    )

    @field_validator("grade", mode='before')
    def validate_grade(cls, value):
        if value == "not relevant":
            return "irrelevant"
        return value

grader_system_prompt_template = """"You are a grader tasked with assessing the relevance of a given context to a query. 
    If the context is relevant to the query, score it as "relevant". Otherwise, give "irrelevant".
    Do not answer the actual answer, just provide the grade in JSON format with "grade" as the key, without any additional explanation."
    """

grader_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", grader_system_prompt_template),
        ("human", "context: {context}\n\nquery: {query}"),
    ]
)


grader_chain = grader_prompt | llm.with_structured_output(Grader, method="json_mode")

query = "symptoms of migraine"
context = retriever.get_relevant_documents(query)

response = grader_chain.invoke({"query": query, "context": context})

/var/folders/bg/7gm4yt417px42myyczdlqf1h0000gn/T/ipykernel_47828/2785852064.py:36: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = retriever.get_relevant_documents(query)


In [10]:
print(response)

grade='relevant'


In [11]:
query = "Treatment of Ulcer"
context = retriever.get_relevant_documents(query)

response = grader_chain.invoke({"query": query, "context": context})
print(response)

grade='irrelevant'


In [12]:
from langchain_core.output_parsers import StrOutputParser
from IPython.core.display import Markdown

rag_template_str = (
    "You are a helpful assistant. Answer the query below based only on the provided context.\n\n"
    "context: {context}\n\n"
    "query: {query}"
)


rag_prompt = ChatPromptTemplate.from_template(rag_template_str)
rag_chain = rag_prompt | llm | StrOutputParser()

query = "What are the symptoms of malaria?"
context = retriever.get_relevant_documents(query)

response = rag_chain.invoke({"query": query, "context": context})

Markdown(response)

According to the provided context, the symptoms of malaria include:

1. High fever
2. Chills
3. Sweating
4. Nausea or vomiting
5. Headache
6. Diarrhea
7. Being very tired (fatigue)
8. Body aches
9. Yellow skin (jaundice)
10. Kidney failure
11. Seizure
12. Confusion
13. Bloody stools
14. Convulsions
15. Death

Additionally, symptoms of serious malaria include:

1. Impaired consciousness
2. Convulsions
4. Difficulty breathing
5. Serious tiredness and fatigue
6. Dark or bloody urine
7. Yellow eyes and skin (jaundice)
8. Abnormal bleeding

In [13]:
from operator import itemgetter

fallback_prompt = ChatPromptTemplate.from_template(
    (
        "You are a friendly medical assistant created by NHVAI.\n"
        "Do not respond to queries that are not related to health.\n"
        "If a query is not related to health, acknowledge your limitations.\n"
        "Provide concise responses to only medically-related queries.\n\n"
        "Current conversations:\n\n{chat_history}\n\n"
        "human: {query}"
    )
)

fallback_chain = (
    {
        "chat_history": lambda x: "\n".join(
            [
                (
                    f"human: {msg.content}"
                    if isinstance(msg, HumanMessage)
                    else f"AI: {msg.content}"
                )
                for msg in x["chat_history"]
            ]
        ),
        "query": itemgetter("query") ,
    }
    | fallback_prompt
    | llm
    | StrOutputParser()
)

fallback_chain.invoke(
    {
        "query": "Hello",
        "chat_history": [],
    }
)

"Hello! I'm happy to assist you with any medical-related questions or concerns you may have. What's on your mind today?"

In [14]:
from langchain_core.runnables import RunnableParallel

class HallucinationGrader(BaseModel):
    "Binary score for hallucination check in llm's response"

    grade: Literal["yes", "no"] = Field(
        ..., description="'yes' if the llm's reponse is hallucinated otherwise 'no'"
    )


hallucination_grader_system_prompt_template = (
    "You are a grader assessing whether a response from an llm is based on a given context.\n"
    "If the llm's response is not based on the given context give a score of 'yes' meaning it's a hallucination"
    "otherwise give 'no'\n"
    "Just give the grade in json with 'grade' as a key and a binary value of 'yes' or 'no' without additional explanation"
)

hallucination_grader_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", hallucination_grader_system_prompt_template),
        ("human", "context: {context}\n\nllm's response: {response}"),
    ]
)


hallucination_grader_chain = (
    RunnableParallel(
        {
            "response": itemgetter("response"),
            "context": lambda x: "\n\n".join([c.page_content for c in x["context"]]),
        }
    )
    | hallucination_grader_prompt
    | llm.with_structured_output(HallucinationGrader, method="json_mode")
)

query = "Symptoms of malaria"
context = retriever.get_relevant_documents(query)
response = """Based on the context provided, the symptoms of malaria include: Impaired consciousness, Convulsions, Difficulty breathing,
Serious tiredness and fatigue, Dark or bloody urine, Yellow eyes and skin (jaundice), Abnormal bleeding, High fever, Chills,
Sweating, Nausea or vomiting, Headache, Diarrhea"""

response = hallucination_grader_chain.invoke({"response": response, "context": context})

In [15]:
response

HallucinationGrader(grade='no')

In [16]:
class AnswerGrader(BaseModel):
    "Binary score for an answer check based on a query."

    grade: Literal["yes", "no"] = Field(
        ...,
        description="'yes' if the provided answer is an actual answer to the query otherwise 'no'",
    )


answer_grader_system_prompt_template = (
    "You are a grader assessing whether a provided answer is in fact an answer to the given query.\n"
    "If the provided answer does not answer the query give a score of 'no' otherwise give 'yes'\n"
    "Just give the grade in json with 'grade' as a key and a binary value of 'yes' or 'no' without additional explanation"
)

answer_grader_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", answer_grader_system_prompt_template),
        ("human", "query: {query}\n\nanswer: {response}"),
    ]
)


answer_grader_chain = answer_grader_prompt | llm.with_structured_output(
    AnswerGrader, method="json_mode"
)

query = "Symptoms of malaria"
# context = retriever.get_relevant_documents(query)
response = """Based on the context provided, the symptoms of malaria include: Impaired consciousness, Convulsions, Difficulty breathing,
Serious tiredness and fatigue, Dark or bloody urine, Yellow eyes and skin (jaundice), Abnormal bleeding, High fever, Chills,
Sweating, Nausea or vomiting, Headache, Diarrhea"""

response = answer_grader_chain.invoke({"response": response, "query": query})

In [17]:
response

AnswerGrader(grade='yes')

In [18]:
from typing import TypedDict
from langchain_core.documents import Document
from langgraph.prebuilt import ToolNode
from langchain_core.tools import Tool
from langchain_core.messages.base import BaseMessage
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_search = TavilySearchResults()
tool_executor = ToolNode(
    tools=[
        Tool(
            name="VectorStore",
            func=retriever.invoke,
            description="Useful to search the vector database",
        ),
        Tool(
            name="SearchEngine", func=tavily_search, description="Useful to search the web"
        ),
    ]
)

class AgentSate(TypedDict):
    """The dictionary keeps track of the data required by the various nodes in the graph"""

    query: str
    chat_history:list[BaseMessage]
    generation: str
    documents: list[Document]


def retrieve_node(state: dict) -> dict[str, list[Document] | str]:
    """
    Retrieve relevent documents from the vectorstore

    query: str

    return list[Document]
    """
    query = state["query"]
    documents = retriever.invoke(input=query)
    return {"documents": documents}

def fallback_node(state: dict):
    """
    Fallback to this node when there is no tool call
    """
    query = state["query"]
    chat_history = state["chat_history"]
    generation = fallback_chain.invoke({"query": query, "chat_history": chat_history})
    return {"generation": generation}


def filter_documents_node(state: dict):
    filtered_docs = list()

    query = state["query"]
    documents = state["documents"]
    for i, doc in enumerate(documents, start=1):
        grade = grader_chain.invoke({"query": query, "context": doc})
        if grade.grade == "relevant":
            print(f"---CHUCK {i}: RELEVANT---")
            filtered_docs.append(doc)
        else:
            print(f"---CHUCK {i}: NOT RELEVANT---")
    return {"documents": filtered_docs}


def rag_node(state: dict):
    query = state["query"]
    documents = state["documents"]

    generation = rag_chain.invoke({"query": query, "context": documents})
    return {"generation": generation}


def web_search_node(state: dict):
    query = state["query"]
    results = tavily_search.invoke(query)
    documents = [
        Document(page_content=doc["content"], metadata={"source": doc["url"]})
        for doc in results
    ]
    return {"documents": documents}

def question_router_node(state: dict):
    query = state["query"]
    try:
        response = question_router.invoke({"query": query})
    except Exception:
        return "llm_fallback"

    if "tool_calls" not in response.additional_kwargs:
        print("---No tool called---")
        return "llm_fallback"

    if len(response.additional_kwargs["tool_calls"]) == 0:
        raise "Router could not decide route!"

    route = response.additional_kwargs["tool_calls"][0]["function"]["name"]
    if route == "VectorStore":
        print("---Routing to VectorStore---")
        return "VectorStore"
    elif route == "SearchEngine":
        print("---Routing to SearchEngine---")
        return "SearchEngine"


def should_generate(state: dict):
    filtered_docs = state["documents"]

    if not filtered_docs:
        print("---All retrived documents not relevant---")
        return "SearchEngine"
    else:
        print("---Some retrived documents are relevant---")
        return "generate"
    
def hallucination_and_answer_relevance_check(state: dict):
    llm_response = state["generation"]
    documents = state["documents"]
    query = state["query"]

    hallucination_grade = hallucination_grader_chain.invoke(
        {"response": llm_response, "context": documents}
    )
    if hallucination_grade.grade == "no":
        print("---Hallucination check passed---")
        answer_relevance_grade = answer_grader_chain.invoke(
            {"response": llm_response, "query": query}
        )
        if answer_relevance_grade.grade == "yes":
            print("---Answer is relevant to question---\n")
            return "useful"
        else:
            print("---Answer is not relevant to question---")
            return "not useful"
    print("---Hallucination check failed---")
    return "generate"

In [19]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentSate)
workflow.add_node("VectorStore", retrieve_node)
workflow.add_node("SearchEngine", web_search_node)
workflow.add_node("filter_docs", filter_documents_node)
workflow.add_node("fallback", fallback_node)
workflow.add_node("rag", rag_node)

workflow.set_conditional_entry_point(
    question_router_node,
    {
        "llm_fallback": "fallback",
        "VectorStore": "VectorStore",
        "SearchEngine": "SearchEngine",
    },
)

workflow.add_edge("VectorStore", "filter_docs")
workflow.add_edge("SearchEngine", "filter_docs")
workflow.add_conditional_edges(
    "filter_docs", should_generate, {"SearchEngine": "SearchEngine", "generate": "rag"}
)
workflow.add_conditional_edges(
    "rag",
    hallucination_and_answer_relevance_check,
    {"useful": END, "not useful": "SearchEngine", "generate": "rag"},
)

workflow.add_edge("fallback", END)

In [22]:
app = workflow.compile(debug=False)
plot = app.get_graph().draw_mermaid_png()

with open("plot.png", "wb") as fp:
    fp.write(plot)

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [23]:
from io import BytesIO
from PIL import Image
from IPython.display import display

# Assuming you have the byte image stream in the 'byte_image' variable
img = Image.open(BytesIO(plot))
display(img)

NameError: name 'plot' is not defined

In [24]:
for token in app.stream({"query": "What are the risk factors of migraines", "chat_history": []}):
    print(token, end="", flush=True)

---Routing to VectorStore---
{'VectorStore': {'documents': [Document(metadata={'description': 'Migraine headaches, also known as migraines, are a type of headache that can cause debilitating pain. Learn more about the types, causes, symptoms, diagnosis, & treatment of migraines.', 'language': 'en', 'source': 'https://www.webmd.com/migraines-headaches/migraines-headaches-migraines', 'title': 'Migraine: Symptoms, Causes, and Treatment'}, page_content='your brain swell, while serotonin causes them to shrink. An imbalance in these chemicals causesinflammationand pain.Migraine Risk FactorsThe American Migraine Foundation estimates that more than 38 million Americans get migraines. Some things may make you more likely to get them:Sex.Women (and those assigned female at birth) have migraines three times more often than men and those assigned male at birth.Age.Most people start having migraine headaches between ages 10 and 40. If you have periods,'), Document(metadata={'description': 'Migraine

In [25]:
response = app.invoke({"query": "What are the symptoms of hypertension", "chat_history": []})
Markdown(response["generation"])

---Routing to SearchEngine---
---CHUCK 1: RELEVANT---
---CHUCK 2: RELEVANT---
---CHUCK 3: RELEVANT---
---CHUCK 4: RELEVANT---
---CHUCK 5: RELEVANT---
---Some retrived documents are relevant---
---Hallucination check passed---
---Answer is relevant to question---



Based on the given context, the symptoms of hypertension are:

1. Headaches
2. Blurred vision
3. Nosebleeds
4. Chest pain
5. Dizziness
6. Fatigue
7. Confusion
8. Nausea
9. Blood spots in eyes
10. Facial flushing
11. Sweating
12. Shortness of breath
13. Heart palpitations
14. Anxiety
15. Vomiting

Note that most people with hypertension don't have any symptoms until it's severe. It's essential to have your blood pressure checked regularly to detect hypertension early on.

In [26]:
import gradio as gr
from uuid import uuid4
from langchain_core.messages import HumanMessage, AIMessage

history = {}
session_id = str(uuid4())

def chat(query):

    # Initialize the chat history for the current session
    if session_id not in history:
        history[session_id] = []

    chat_history = history[session_id]

    # Invoke the app with the current query and chat history
    result = app.invoke({"query": query, "chat_history": chat_history})

    # Separate the response from the retrieved documents
    response = result["generation"]
    documents = result["documents"]

    # Add the current exchange to the chat history
    chat_history.extend([HumanMessage(content=query), AIMessage(content=response)])

    if not documents:
        return response, documents

    documents = [
        f"{doc.page_content}\nsource: {doc.metadata['source']}" for doc in documents
    ]

    return response, "\n\n".join(documents)

# Create the Gradio interface
demo = gr.Interface(
    fn=chat,
    inputs=gr.Textbox(label="Question"),
    outputs=[gr.Textbox(label="Response"), gr.Textbox(label="Retrieved Documents")],
    title="Cortex Chatbot",
    description="Ask a health-related query and the chatbot will generate a response using Retrieval Augmented Generation.",
)

if __name__ == "__main__":
    demo.launch(share=True, inline=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://09d0786058a440e354.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---Routing to SearchEngine---
---CHUCK 1: RELEVANT---
---CHUCK 2: RELEVANT---
---CHUCK 3: RELEVANT---
---CHUCK 4: RELEVANT---
---CHUCK 5: RELEVANT---
---Some retrived documents are relevant---
---Hallucination check passed---
---Answer is relevant to question---

---Routing to SearchEngine---
---CHUCK 1: RELEVANT---
---CHUCK 2: RELEVANT---
---CHUCK 3: RELEVANT---
---CHUCK 4: RELEVANT---
---CHUCK 5: RELEVANT---
---Some retrived documents are relevant---
---Hallucination check passed---
---Answer is relevant to question---

---Routing to SearchEngine---
---CHUCK 1: RELEVANT---
---CHUCK 2: RELEVANT---
---CHUCK 3: RELEVANT---
---CHUCK 4: RELEVANT---
---CHUCK 5: RELEVANT---
---Some retrived documents are relevant---
---Hallucination check passed---
---Answer is relevant to question---

